# Comparator computation

This computes the Euclidean distance for each establishment from a base establishment of a consistent type (Academies, Maintained schools, SEN). Each establishment is given a weight against every other establishment and the top 60 for each establishment creates the comparator set for that establishment. This is repeated for all establishments in the establishment types set and then finally across all establishments across all establishment types.

In [1]:
import numpy as np
import pandas as pd
import comparator_sets as comparators
import glob 
import os

# Create and clean directory
from pathlib import Path
Path("output/comparator-sets").mkdir(parents=True, exist_ok=True)

files = glob.glob("output/comparator-sets/*")
for f in files:
    os.remove(f)

# Prepare Academy and School Data

Here we prepare the academy and maintained school data by filling in missing values in NumberOfPupils, % Free School Meals and, % Sen with the mean (at this time). 

In [2]:
academy_data = comparators.prepare_data(pd.read_csv("output/pre-processing/academies.csv"))
ms_data = comparators.prepare_data(pd.read_csv("output/pre-processing/maintained_schools.csv", low_memory=False))
all_schools = pd.concat([academy_data, ms_data])

# Compute the pupil and building comparators

This creates the comparators sets across both academy and maintained schools

In [3]:
pupil_comparators = comparators.compute_comparator_matrix(all_schools, comparators.compute_pupils_comparator)
building_comparators = comparators.compute_comparator_matrix(all_schools, comparators.compute_buildings_comparator)

Save to disk

In [4]:
import pickle
with open('output/comparator-sets/pupil_comparators.pkl', 'wb') as outfile:
     pickle.dump(pupil_comparators, outfile, protocol=pickle.HIGHEST_PROTOCOL)
     
with open('output/comparator-sets/building_comparators.pkl', 'wb') as outfile:
     pickle.dump(building_comparators, outfile, protocol=pickle.HIGHEST_PROTOCOL)

Below is an example of extracting a school by name to show how the data structures work

In [5]:
target_school = 'Glebe Primary School'

comparator_set = comparators.get_comparator_set_by(lambda s: s['EstablishmentName'] == target_school, all_schools, pupil_comparators)
comparator_set[['URN']]

,URN
0,145110
3,117156
6,136967
7,117400
16,142036
1,118393
2,101601
4,134681
5,132251
8,105766


# Example using a custom comparator set

The example below selects a set of URN's based on a defined filter. And the 

In [7]:
# Custom specify some selection criteria for schools.
target_urn = 145110
custom_comparator_schools = all_schools[(all_schools['PFI School'] == 'PFI School') | (all_schools.index == target_urn)]
custom_comparators = comparators.compute_custom_comparator('PFI Comparator', custom_comparator_schools, comparators.compute_pupils_comparator)
cust_set = comparators.get_comparator_set_by(lambda s: s.index == target_urn, all_schools, custom_comparators, is_custom=True, comparator_key='PFI Comparator')

cust_set[['URN', 'Percentage SEN', 'Percentage Free school meals']]

,URN,Percentage SEN,Percentage Free school meals
0,145110,5.164319,18.7
58,145856,0.613497,29.4
59,139491,1.867220,33.0
1,140242,2.347418,17.6
2,140362,2.449889,20.9
3,138160,2.163462,16.1
4,142807,1.398601,16.1
5,137012,3.067485,16.0
6,139113,1.923077,21.2
7,148019,3.537736,22.8


# Computing RAG

RAG 

,URN,Company Registration Number,Academy Trust UPIN,Academy UKPRN,Academy UPIN,Valid to,Date left or closed if in period,Number of Academies in Trust,Number of pupils,LA (code),...,"Total Expenditure E01 to E29 and E31 to E32 minus I9, I10, I16 and I17",Total Expenditure E01 to E32,Revenue Reserve B01 plus B02 plus B06,In-year Balance Total Income (I01 to I18) minus Total Expenditure (E01 to E32),% of pupils known to be eligible for free school meals (Performa,School Balance,School Financial Position,Partial Years Present,Did Not Submit,Distance
count,30.000000,3.000000e+01,30.000000,3.000000e+01,30.000000,0.0,0.0,30.000000,30.000000,30.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.000000
mean,141274.666667,8.041527e+06,136419.500000,1.005824e+07,129371.633333,NaN,NaN,16.033333,372.266667,780.400000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.047423
std,2692.629907,9.505672e+05,1253.082537,5.380045e+03,8840.358334,NaN,NaN,13.220891,86.725064,234.310522,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.025316
min,136849.000000,6.182612e+06,134889.000000,1.003924e+07,119849.000000,NaN,NaN,1.000000,167.000000,320.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
25%,139207.500000,7.695641e+06,135294.250000,1.005826e+07,123050.500000,NaN,NaN,6.000000,330.500000,861.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.032868
50%,140703.000000,8.033790e+06,135947.000000,1.005936e+07,128684.500000,NaN,NaN,9.000000,415.500000,879.500000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.043422
75%,142752.500000,8.628471e+06,137509.500000,1.006055e+07,132196.500000,NaN,NaN,29.000000,431.500000,941.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.061190
max,148019.000000,1.048281e+07,139699.000000,1.006419e+07,162519.000000,NaN,NaN,44.000000,459.000000,941.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.113490
